<a href="https://colab.research.google.com/github/20hnu/machine_learning/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import random


##Simple Rule Based ChatBot

In [2]:

class RuleBot:
  negative_response = ('no','nah','nope','naw','not a chance','sorry')
  exit_commands = ("quit", "pause","exit","goodbye","bye","later")
  random_questions = (
      "Why are you here?",
      "Are there many humans like you?",
      "What do you consume for sustenance?",
      "Is there intelligent life on this planet?",
      "Does Earth have a leader?",
      "What planets have you visited?",
      "What technology do you have on this planet?"
  )

  def __init__(self):
    self.alienbubble = {'describe_planet_intent': r'.*\s*your planet.*',
                        'answer_why_intent': r'why\sare.*',
                        'about_intellipat': r'.*\s*intellipaat'
                        }
  def greet(self):
    self.name = input("What is your name?\n")
    will_help = input(
        f"Hi {self.name}, I am Rule-Bot. Will you help me learn about your planet?\n")
    if will_help in self.negative_response:
      print("Ok, have a nice Earth day!")
      return
    self.chat()

  def make_exit(self, reply):
    for command in self.exit_commands:
      if reply == command:
        print("Okay, have a nice Earth day!")
        return True

  def chat(self):
    reply = input(random.choice(self.random_questions)).lower()
    while not self.make_exit(reply):
      reply = input(self.match_reply(reply))

  def match_reply(self, reply):
    for key, value in self.alienbubble.items():
      intent = key
      regex_pattern = value
      found_match = re.match(regex_pattern, reply)
      if found_match and intent == "describe_planet_intent":
        return self.describe_planet_intent()
      elif found_match and intent == "answer_why_intent":
        return self.answer_why_intent()
      elif found_match and intent == "about_intellipat":
        return self.about_intellipat()
    if not found_match:
      return self.no_match_intent()

  def describe_planet_intent(self):
    responses = ("My planet is a utopia of diverse organisms and species. \n",
                 "I am from Opidipus, the capital of the Wayward Galaxies. \n")
    return random.choice(responses)

  def answer_why_intent(self):
    responses = ("I come in peace\n", "I am here to collect data on your planet and its inhabitants\n",
                 "I heard the coffee is good\n")
    return random.choice(responses)

  def about_intellipat(self):
    responses = ("Intellipaat is world's largest professional educationsl company\n","Intellipaat will make you learn concepts in the way never to be forgotten\n",
                 "Intellipaat is where your career and skill grows\n")
    return random.choice(responses)

  def no_match_intent(self):
    responses = ("Please tell me more.\n", "Tell me more!\n", "why do you say that?\n","I see. Can you elaborate?\n",
                 "Interesting. Can you tell me more?\n","I see. How do you think?\n","why?\n","How do you think i feel when you say that?\n")
    return random.choice(responses)


AlienBot = RuleBot()
AlienBot.greet()

What is your name?
bye
Hi bye, I am Rule-Bot. Will you help me learn about your planet?
bye
Why are you here?bye
Okay, have a nice Earth day!


##NLP based ChatBot

In [3]:
import numpy as np
import nltk
import string
import random

In [4]:
f = open("/content/report.txt",'r', errors="ignore")
raw_doc = f.read()

In [5]:
raw_doc = raw_doc.lower()
nltk.download('punkt') # Using the Punkt toeknizer
nltk.download('wordnet') #Using the wordnet dictionary
nltk.download('omw-1.4')

sentence_tokens = nltk.sent_tokenize(raw_doc)
word_tokens = nltk.word_tokenize(raw_doc)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [6]:
len(sentence_tokens)

466

In [7]:
len(word_tokens)

8375

In [8]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]

remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

In [9]:
greet_inputs = ('hello','hi','whassup','how are you?')
greet_responses = ('hi','hey', 'Hey There!','There there!!')
def greet(sentence):
  for word in sentence.split():
    if word.lower() in greet_inputs:
      return random.choice(greet_responses)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def response(user_response):
  robo1_response = ''
  TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words = "english")
  tfidf = TfidfVec.fit_transform(sentence_tokens)
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if(req_tfidf == 0):
    robo1_response = robo1_response + "I am sorry. Unable to understand you!"
    return robo1_response
  else:
    robo1_response = robo1_response + sentence_tokens[idx]
    return robo1_response


In [11]:
flag = True
print("Hello! I am the Retrieval Learning Bot. Start typing your text after greeting to talk to me. For ending convo type bye!")
while(flag==True):
  user_response = input()
  user_response = user_response.lower()
  if (user_response != 'bye'):
    if (user_response == "thank you" or user_response == 'thanks'):
      flag = False
      print('Bot: You are Welcome..')
    else:
      if(greet(user_response) != None):
        print('Bot '+ greet(user_response))
      else:
        sentence_tokens.append(user_response)
        word_tokens = word_tokens + nltk.word_tokenize(user_response)
        final_words = list(set(word_tokens))
        print('Bot ', end = '')
        print(response(user_response))
        sentence_tokens.remove(user_response)
  else:
    flag = False
    print("Bot: Goodbye!")

Hello! I am the Retrieval Learning Bot. Start typing your text after greeting to talk to me. For ending convo type bye!
Hi
Bot hey
what is chatbot
Bot 

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


"dbpedia chatbot".
can you tell me about turing test
Bot chatbot competitions focus on the turing test or more specific goals.
Universe
Bot I am sorry. Unable to understand you!
bye
Bot: Goodbye!
